# My Pagerank 

## How?
Pagerank implementation done from scratch using the *power iteration method*. In this method i tried to use specific-platform frameworks to speed up the computation
## Why? 

# Code


## Platform identification
This first section of code it's used to identify which platform we are using (generic Cpu, Apple Silicon or Nvidia GPU using cuda if present)

In [37]:
import platform
import sys
import numpy as np
from scipy import sparse
import networkx as nx


def get_platform():
    """
    Detects hardware in use and returns (platform_name, library_to_use ).
    """
    if sys.platform == "darwin" and platform.processor() == "arm":
        try:
            import mlx.core as mx
            return "mlx", mx
        except ImportError:
            pass
    try:
        import torch
        if torch.cuda.is_available():
            return "cida", torch
    except ImportError:
        pass
    return "cpu", np

In [38]:
if __name__ == "__main__":
    print(get_platform()[0])

mlx


## PageRank

In [ ]:
def get_prscores (graph_obj, alpha = 0.85, tol= 1e-5, max_iter =1000):
    '''
    Compute PageRank Score 
    graph is the loaded pickle file 
    '''
    
    # Extract Adjacency Matrix
    if hasattr(graph_obj, 'adjacency'):
        node_labels = list(graph_obj.nodes())
        # Convert to Scipy CSC matrix 
        adj = nx.to_scipy_sparse_array(graph_obj, format='csc', dtype=np.float32)
    else:
        adj = graph_obj.tocsc()
        node_labels = list(range(adj.shape))
    
    #number of nodes 
    n_nodes = adj.shape[0]
    
    if __name__ == "__main__":
        display(n_nodes)
    
    #we now can calcualate the out degree of each node (that given the adjaceny matrix 
    # is simply the sum along colums )
    out_degrees = np.array(adj.sum(axis=0)).flatten()
    is_sink = (out_degrees == 0)                 #another check if a node is a sink
    
    # Normalize transition probabilities: P_ij = 1 / out_degree(j) (ignoring sinks)
    norm_out_degrees = np.where(is_sink, 1.0, out_degrees)
    adj.data = adj.data / norm_out_degrees[adj.indices]
    
    platform_name, engine = get_platform()
    
    print("for now we always use cpu ")
    
    #we create the pagerank vector (initialized as every node as probability 1/n)
    pr = np.full(n_nodes, 1.0/n_nodes)
    teleport_const = (1.0 - alpha) / n_nodes
    for i in range(max_iter):
        
        pr_next = adj.dot(pr)
        sink_mass = np.sum(pr[is_sink])
        pr_next = alpha*(pr_next +sink_mass / n_nodes)+teleport_const
        
        # check if maximum difference is lower than tol, if yes not much imprvement so we break
        if np.linalg.norm(pr_next - pr,1)<tol:
            break
        #else we update the pagerank vector 
        pr = pr_next
        
        prscores = dict(zip(node_labels, pr))
        return prscores